## GRAY IMAGES

In [ ]:
from PIL import Image
import pandas as pd
import os

In [ ]:
anot_path = "./data/attention.csv"
images_path = "./data/datasets/"

gray_images_path = "./data/grayimage"

In [ ]:
df = pd.read_csv(anot_path)
df.columns = ['filename', 'score']
df['score'].value_counts()

In [ ]:
save_path = "data/gray_data.csv"

data = {
    'width': [],
    'height': [],
    'file_path': [],
    'score': []
}

if not os.path.exists(gray_images_path):
    os.makedirs(gray_images_path)

for item, score in zip(df['filename'].tolist(), df['score'].tolist()):
    file_path = os.path.join(images_path, item)
    gray_path = os.path.join(gray_images_path, item)

    img = Image.open(file_path).convert("L")
    
    img.save(gray_path)
    
    data['width'].append(img.size[0])
    data['height'].append(img.size[1])
    data['file_path'].append(gray_path)
    data['score'].append(score)

In [ ]:
pd.DataFrame(data).to_csv(save_path, index=False)

## BLUR CANNY CONTOUR 

In [ ]:
import pandas as pd
from PIL import Image
import random
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# anomali data 69, 5, 36
# rd_idx = 5

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

def Proccess(df = df, idx = None, biar_rapih = False):
    if idx == None:
        idx = random.randint(0, len(df['file_path'].tolist()))
    
    sample_image_name = df['file_path'].tolist()[idx]
    score = df['score'].tolist()[idx]

    img_np = np.array(Image.open(sample_image_name))
    
    if not biar_rapih:
        # gausah didebug ini [biar apa biarin | pusing gw njir ngeliatnya]
        print(f"image idx: {idx} | score: {score} | filename: {sample_image_name}")
        print(f"image size: {img_np.shape}")

    blank = np.zeros(shape=img_np.shape, dtype='uint8')

    gaussBlur = cv.GaussianBlur(img_np, (175, 175), 0.3)
    canny = cv.Canny(gaussBlur, 100, 100)

    _, tresh = cv.threshold(img_np, 125, 255, cv.THRESH_BINARY)
    _, blurtresh = cv.threshold(gaussBlur, 125, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(blank, contours, -1, 255, 1)
    
    images = [img_np, gaussBlur, canny, tresh, blank]
    titles = ['Gray', 'Gaussian Blur', 'Canny', 'Threshold', 'Contours Drawn']
    
    return images, titles

def ShowImage(images, titles):
    
    plt.figure(figsize=(15, 8))

    for i in range(len(images)):
        plt.subplot(2, 3, i + 1)  # 2 baris, 3 kolom
        if len(images[i].shape) == 2:  # grayscale
            plt.imshow(images[i], cmap='gray')
        else:  # RGB image
            plt.imshow(cv.cvtColor(images[i], cv.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
max_w = max(df['width'].tolist())
min_w = min(df['width'].tolist())

max_h = max(df['height'].tolist())
min_h = min(df['height'].tolist())

print(f"max-width: {max_w} | min-width: {min_w} | max-height: {max_h} | min-height: {min_h}")

In [ ]:
for i in range(5):
    images, titles = Proccess()
    ShowImage(images, titles)

## LOAD DATA INTO NDARRAY

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

df.describe()

# mean width = 624
# mean height = 208

# biar 2^ pake width = 512 (2**9)
# biar 2^ pake height = 256 (2**8)

In [ ]:
def resize(np_img, w = 512, h = 256):
    return cv.resize(np_img, dsize=(w, h), interpolation=cv.INTER_LINEAR)

In [ ]:
gray_np = [] # gray images
gaussBlur_np = []
canny_np = []
tresh_np = []
contour_np = []

# load to array
for i in range(len(df['file_path'].tolist())):
    '''
    images = [
        img_np, gaussBlur, canny, tresh, blank]
        [gray, gaussblur(gray), canny(gray), tresh(gray), contour(grey)
    ]
    '''
    
    images, _ = Proccess(idx=i, biar_rapih=True)
    # print(type(resize(images[0])))
    # print(resize(images[0]))
    # print(resize(images[0]).shape)
    # break
    gray_np.append(resize(images[0]))
    gaussBlur_np.append(resize(images[1]))
    canny_np.append(resize(images[2]))
    tresh_np.append(resize(images[3]))
    contour_np.append(resize(images[4]))

# transform into 
gray_np = np.array(gray_np)
gaussBlur_np = np.array(gaussBlur_np)
canny_np = np.array(canny_np)
tresh_np = np.array(tresh_np)
contour_np = np.array(contour_np)

In [ ]:
print(gray_np.shape, gaussBlur_np.shape, canny_np.shape, tresh_np.shape, contour_np.shape)

# pake dah mau pake yang mana

## TRY ML MODEL [alex]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [ ]:
X = tresh_np.reshape(tresh_np.shape[0], -1)
X = X / 255
y = np.array(df['score']).reshape(-1, 1)

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
type(X_train[0]), X_train[0]

In [ ]:
type(y_train[0]), y_train[0]

In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 8, 16, 64],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [5, 15, 20],
    'bootstrap': [True, False]
}

random_forest = RandomForestClassifier(random_state=42)
random_forest

In [ ]:
random_search = RandomizedSearchCV(
    estimator=random_forest,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

In [ ]:
best_params = random_search.best_params_
best_estimators = random_search.best_estimator_
best_score = random_search.best_score_

print(f"best scores: {best_score} \nbest parameters: {best_params}")

In [ ]:
best_estimators